# Pandas Tutorial

In this lab session, we'll go through some useful ways to use [Pandas](https://pandas.pydata.org/) for data science. As is the case with the previous tutorial, this is NOT a comprehensive Pandas tutorial, but rather a collection of tips and useful fucntions that I expect you to use for the labs, exams, and courseworks of the course.

* This is a self-paced tutorial, but make sure to take some time on this week to get familiar with the concepts.
* If you find any part of this tutorial complicated, please prepare a list of questions for the beginning of the next lab.
* The usual: the canonical answer to any of you questions is probably contained in the [Pandas User Guide](https://pandas.pydata.org/docs/user_guide/index.html) and [Documentation](https://pandas.pydata.org/docs/)
* Similar to NumPy, Pandas is one of the essential tools for data science with Python, and if you're serious about data science as a career, you should take as long as you need to become proficient with the tool. I believe the best place to start is the [10 minutes to pandas](https://pandas.pydata.org/docs/user_guide/10min.html) tutorial.

## Table of Contents

1. [Series](#Series)
2. [DataFrames](#DataFrames)
3. [Indexing](#Indexing)
4. [Long vs narrow data](#Long-vs-narrow-data)
5. [Useful patterns](#Useful-patterns)
6. [Copies and Views](#Copies-and-Views)


In [ ]:
# Only run this cell if you're using Google Colab

!git clone https://github.com/torresmateo/fgv-class-2022.git
!cp -r fgv-class-2022/images .
!cp -r fgv-class-2022/tutorials .
!cp -r fgv-class-2022/data .

In [ ]:
import numpy as np
import pandas as pd
from tutorials.utils import *

## Series

Pandas, like NumPy, provides most of its functionality by providing data structures and functions that operate on such structures. The first data structure is the [`Series`](https://pandas.pydata.org/docs/reference/api/pandas.Series.html). 

At the very core, a Series is a one-dimensional NumPy ndarray that supports axis labels. This immediately offers some advantages when compared to a ndarray. To see some of these advantages, let's create some Series below

In [ ]:
# let's start by creating an alphabetic index for our ndarray
index = list(alpha_range(5))

# then, let's create a one-dimensional ndarray that will hold the data for our Series
data = np.arange(5)

print(f"index: {index}")
print(f"data: {data}")

# Now we can create a that uses the index to name each of the componens of our one-dimensional ndarray
s1 = pd.Series(data, index=index)

print(f"Series:\n{s1}")

# By default the index is simply a range that enumerates each component
rng = np.random.default_rng(0)
s2 = pd.Series(rng.random(5))

print(f"Series with default index:\n{s2}")

notice how the `Series` object not only holds the data and the index, but also some metadata such as the type.

### `Series` as a `ndarray`

`Series` objects are ndarray-like. This means that most NumPy operators will operate on a `Series`, including the slicing operations:

In [ ]:
print(f"s1:\n{s1}")

# Notice how slicing this series to reverse it works exactly like it does for a ndarray,
# but it reverses also the index. This is extremely useful for keeping sorted indices consistent
# with the data. 
print(f"\ns1[-1::-1]:\n{s1[-1::-1]}")

# It also support the NumPy advanced slicing
# In this example, you can see also that the index is not necessarily unique.

print(f"\ns1[[0,1,0,3]]:\n{s1[[0,1,0,3]]}")

### `Series` as a Python `dict`

A convenient way to access and modify values on a `Series` is using the Python `dict` syntax:

In [ ]:
print(f"s1[\"a\"]: {s1['a']}")

s1["a"] = 20

print(f's1["a"]: {s1["a"]}')

print("\nCase with non-unique indices\n")


# We create now a series with distinct values, but repeated indices
s3 = pd.Series(np.arange(5), index=["a", "b", "a", "b", "a"])

print(f"Original Series s3:\n{s3}")

s3["a"] = 0

print(f'\nAfter modifying the "a" key s3:\n{s3}')

# Notice also that using repeated indices will return all values that match the selected index
print(f'\ns3["b"]:\n{s3["b"]}')


### `Series` operations

Like `ndarrays`, `Series` offers a large collection of functions and operators for a single or multiple series:

In [ ]:
print(f"s1:\n{s1}\n")
print(f"s2:\n{s2}\n")
print(f"s3:\n{s3}\n")

print(f"s1.mean(): {s1.mean()}\n")
print(f"s2.median(): {s2.median()}\n")
print(f"pd.concat([s1, s2]):\n{pd.concat([s1, s2])}\n")

# Notices this special case, where s1 and s3 share only some of the indices. 
# The rules that establish how the operators will be mapped is known as "alignment", 
# and we'll see it later in the tutorial
print(f"s1 + s3:\n{s1 + s3}")

## DataFrames

While `Series` is the basic data structure of Pandas, by far the most used data structure is the Pandas [`DataFrame`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html).

The easiest (although incorrect) way to think about a `DataFrame` is a 2-dimensional `Series` object. This is mostly because a `DataFrame` can be heterogeneous on the types of each of the series that compose it.

For those used to dealing with spreadsheets and SQL tables, a `DataFrame` will feel quite familiar. However, much like a `ndarray` is not a mathematical vector, a `DataFrame` is NOT a 2-dimensional ndarray, nor a SQL table, nor an Excel spreadsheet, even though many operations that apply to those objects apply to the `DataFrame` in very similar ways. A `DataFrame` is a unique data structure that inherits many properties of `Series` and `ndarray`.

A special characteristic of a `DataFrame` is that its two dimensions are _labeled_, meaning you have a `Series`-like index on the rows and the columns of the data.

In [ ]:
# Let's create our first DataFrame as a dictionary of Series
s1 = pd.Series(np.arange(5), index=alpha_range(5))
s2 = pd.Series(np.arange(3), index=alpha_range(3))
s3 = pd.Series(np.arange(4), index=alpha_range(4))

print(f"s1:\n{s1}\n")
print(f"s2:\n{s2}\n")
print(f"s3:\n{s3}\n")

data = {
    'one': s1, 
    'two': s2, 
    'three': s3
}
# we simply pass this dictionary to the constructor, and we get our first DataFrame
df = pd.DataFrame(d)

# if we print it, we get a text table
print(df)

# Because we're in a notebook, we can inspect the variable "as is" and get a web-friendly table
df

### Data Alignment in Pandas

The `DataFrame` we created above is convenient to explain the concept of data alignment in Pandas.

Notice how `s1` is the only `Series` with 5 indices (`a` to `e`). `s2` and `s3` share 3, and 4 of those letter respectively.

Because the underlying representation a `DataFrame` must have _something_ on the coordinates where data was not provided, it will fill this empty spaces with the default NumPy representation for missing data: `np.nan` (printed as `NaN`)

Notice that the `int64` data type was changed to something else for columns `two` and `three`, this is because `np.nan` is defined as a `float` value in NumPy, and therefore all the Series was cast into `float64`:

In [ ]:
df.dtypes

### The `Series` in a `DataFrame`

let's dissect the `DataFrame` s bit more, and see if we can reveal some more of the underlying data structure:

In [ ]:
print(f'df["one"]:\n{df["one"]}\n')
print(type(df["one"]))

We can see that a column is actually a `Series`, let's see one of the rows:

In [ ]:
print(f'df.loc["d"]:\n{df.loc["d"]}\n')
print(type(df.loc["d"]))

Again, when we extract a row, we get a `Series`. Notice, however that the `3` on the `one` index of this `Series` was cast into `3.0` (`int64` -> `float64`)

## Indexing

We've seen some Numpy-like indexing above, which are the most intuitive and straightforward ways to select subsets of the data in Pandas. 

During the course, we won't need much advanced indexing, but it's very important that you read the [User Guide](https://pandas.pydata.org/docs/user_guide/indexing.html), as the flexibility of data selection in Pandas can become very complex an confusing. Here, I simply provide some useful examples that should be useful for the course.

To start, let's load a dataset of Clinical Trials downloaded in 2020 from [ClinicalTrials.gov](https://clinicaltrials.gov/) and that were related to COVID-19

In [ ]:
clinical_trials = pd.read_table('data/clinical_trials-raw.tsv')
clinical_trials

So, at a glance, we know we have 4645 Clinical Trials, and we can see some of the columns on the table representation. but this does not tell us very useful things so far. Let's inspect the `DataFrame` further. Let's get an idea of all the columns and see if we can make some useful selections to answer some questions about the data

In [ ]:
clinical_trials.dtypes

We observe that most of the columns are strings (which are automatically cast to `object`. Some of these columns seem to have been cast erroneusly thoug, such as `Age`, which could be an integer. Let's inspect that column:

In [ ]:
clinical_trials["Age"]

This explains it, it's not a single number, but a description of a _range_ of ages of the patients involved. We also see that there is a categorical value between parentheses in the field. If we can extract this into a new Series, we could visualize the Age distribution for COVID-related clinical trials. Let's extract these values, and count the number of occurences of each:

In [ ]:
clinical_trials["Age category"] = clinical_trials["Age"].str.split('(').str[1].str.split(')').str[0]

# we can get the values to the console
print(clinical_trials["Age category"].value_counts())

# or use a librart to make a chart with the data, which could be more intuitive for visual comparison
sns.countplot(y=clinical_trials["Age category"])

This is interesting, simply by exploring the data, we can already conclude that around April 2020, most clinical trials were targetting Adults and Older Adults rather than children. The analysis is not as clean as it could be, because some of the categories are still mixed together. I leave the task of doing further cleaning of this field to you as an exercise (such cleaning was part of the Python Assessment at the beginning of the course)

Let's pick another column and do a similar analysis:

In [ ]:
sns.countplot(y=clinical_trials["Phases"],
              order=clinical_trials["Phases"].value_counts().index
             )

## Long vs narrow data

The previous dataset was in record, or "wide" format. That format is intuitive because we can mentally map a row to a "record" with properties.
This can get complicated however, if a particular field should contain a list of values instead of a single value. The previous dataset decided to 
use a secondary (and I've tertiary, and even quaternary) level of separation, with another separation character.

A popular alternative, due to its simplicity, is the Long format, which spreads a single record across multiple rows, and this allows us to avoid dealing with so many separators... let's look at one example. I'm going to load a dataset in long format, and I will create a wide datastet by [pivoting](https://pandas.pydata.org/docs/user_guide/reshaping.html) it (although in this case, with some data loss).

In [ ]:
drugbank = pd.read_pickle('data/drugbank.pkl')
# Let's do a simple (but lossy) pivot of the data
db_long = drugbank.pivot_table(index='DrugBank ID', columns='variable', values='value', aggfunc=min)

In [ ]:
drugbank

In [ ]:
db_long

We can see that `drugbank` has only 3 columns, and that the values of the `DrugBank ID` and `variable` columns are repeated. This, however, is very useful to quickly aggregate statistics in Pandas, without going through the cleaning process of separating fields with multiple values. For instance, let's make a count plot of the groups of the drug:

In [ ]:
# We start by making a boolean mask to select only the rows with the property we want
condition = drugbank['variable'] == 'Group'
sns.countplot(y=drugbank.loc[condition, 'value'])

Because a drug can be in multiple groups simultaneously, getting the same result from the wide format is way more difficult (actually impossible if we use the pivot I did above, because we kept only one of the groups for each drug). 

Now, let's do a more complex calculation. Let's extract the distribution of drug targets. 

In [ ]:
condition = drugbank['variable'] == 'Target'

# let's make a wider figure, so it's readable
fig, ax = plt.subplots(figsize=(25, 5))
# we make the plot (notice the call to value_counts
sns.countplot(x=drugbank.loc[condition, 'DrugBank ID'].value_counts(), ax=ax)

# set a logarithmic scale for the plot
ax.set_yscale('log') 

# set sensible labels for the axis
ax.set_ylabel('count (log)') 
ax.set_xlabel('Target counts') 

plt.show()
plt.close('all')

## Useful patterns

The usage of Pandas will be unique for every project and particular characteristics of the dataset you're dealing with on that context.

However, there are some patterns that will be useful in most cases. This is a very non-comprehensive set of examples.

### Loading from non-standard formats

Often, it is impossible to get a "clean" dataset where every field is separated perfectly, and very often you will need to write a little parser for the data.

Let's have a look at the file contained in `data/example_data.txt`

In [ ]:
with open('data/example_data.txt') as f:
    print(f.read())

The header describes the format of the file, and it's pretty simple to understand. 

However, we'll have some trouble attempting to read this file using the built-in functions in Pandas:

In [ ]:
pd.read_table('data/example_data.txt', comment='!', sep=' ')

A file with this format is famously (or infamously) the output of HMMER, a tool for aligning protein sequences based on Hidden Markov Models. The solution in this case, is to parse the lines ourselves, and build a dictionary or list of dataclasses to populate out `DataFrame`. For simplicity, I will write a simple parser that does not require a dataclass:

In [ ]:
# our simple parser splits the line up to 4 times (creating 5 pieces)
def parse_line(l):
    fields = l.strip().split(' ', 4)
    for i in range(4):
        fields[i] = float(fields[i])
    return fields

columns = ['field1', 'field2', 'field3', 'field4', 'field5']
data = {c:[] for c in columns}

skip_header = True
with open('data/example_data.txt') as f:
    for line in f:
        if line[0] == '!':
            continue
        if skip_header:
            skip_header = False
            continue
        fields = parse_line(line)
        for c, f in zip(columns, fields):
            data[c].append(f)
df = pd.DataFrame(data)
df

## Copies and Views

Like NumPy, the underlying assumption for Pandas data structures is that you get a view of the object when you slice it, rather than a copy. In Pandas, however, we often use very complex selection patterns, that force Pandas to return a copy instead. 

A very common mistake is to "update" some values on a copy instead of a view, and assume that the original dataset is being modified. Dealing with this can be quite tricky, but fortunately, we won't need to do such kind of data cleaning during the course.

Nevertheless, I think it's quite important that you are aware of this, and to point you to the [relevant documentation](https://pandas.pydata.org/docs/user_guide/indexing.html#returning-a-view-versus-a-copy)